# Wikipedia Events Database
---
  
**Goal:** create a database for the events I retrieve from Wikipedia and write python scripts for CRUD-ing data in the database.   
  
  
**Notes:** 
- this is a supporting notebook for the TADS_Wikipedia_this_day_in_history project  

### Preliminary schema

In [2]:
%%html 
<img src = '../../images/img_ss_database_schema_01_01feb21.jpg' width='70%'>

### TO-DO:
---  
  
- [x] create database  
- [x] create tables  
- [x] picture of the database schema
- [x] code for CRUD-ing data in the database
    - get_day_data
        - [x] update wiki_day_data_log
        - [x] update wiki_event
        - [x] update wiki_link
        - [x] update wiki_link_usage
    - get_link_data
        - [x] update wiki_link_data_log
        - [x] update wiki_link_size
        - [x] update wiki_link_page_views
        - [x] update wiki_image <small>*</small>
        - [x] update wiki_image_usage <small>*</small>
    - get_image_data
        - [x] update wiki_license
        - [x] update wiki_user
        - [x] wiki_image_info  
  
<small>* These are image tables but are updated in this step only with data that shows the relation between wiki_link and wiki_image. The image data retrieved from Wikipedia API is updated in step three (get_image_data)</small>  

In [1]:
import sqlite3
import re
import config
import datetime as dt
import numpy as np

In [3]:
DATABASE_FILE = config.DATABASE_FILE
DOE = config.DOE

## Create database and tables

In [4]:
with sqlite3.connect(DATABASE_FILE) as conn:
    # initialize cursor
    c = conn.cursor()
    
    # create table wiki_day_data_log
    c.execute('''CREATE TABLE IF NOT EXISTS wiki_day_data_log 
                (day_data_log_id INTEGER PRIMARY KEY,
                 doe TEXT,
                 day INT,
                 month INT,
                 status_code TEXT,
                 day_soup BLOB,
                 no_event_for TEXT)''')
    
    # create table wiki_link
    c.execute('''CREATE TABLE IF NOT EXISTS wiki_link 
                (link_id INTEGER PRIMARY KEY,
                 doe TEXT,
                 link_url TEXT UNIQUE)''')
    
    # create table wiki_event
    c.execute('''CREATE TABLE IF NOT EXISTS wiki_event 
                (event_id INTEGER PRIMARY KEY,
                 doe TEXT,
                 day INT,
                 month INT,
                 year INT,
                 bc_ad TEXT,
                 bc_ad_note TEXT,
                 event_type TEXT,
                 event_description TEXT,
                 event_category TEXT,
                 event_first_link_id INT,
                 event_links_list TEXT,
                 FOREIGN KEY (event_first_link_id) REFERENCES wiki_link(link_id))''')
    

    # create table wiki_link_usage
    c.execute('''CREATE TABLE IF NOT EXISTS wiki_link_usage 
                (link_usage_id INTEGER PRIMARY KEY,
                 doe TEXT,
                 link_id INT,
                 event_id INT,
                 is_first_link INT,
                 FOREIGN KEY (link_id) REFERENCES wiki_link(link_id),
                 FOREIGN KEY (event_id) REFERENCES wiki_event(event_id))''')
    
    # create table wiki_link_data_log
    c.execute('''CREATE TABLE IF NOT EXISTS wiki_link_data_log 
                (link_data_log_id INTEGER PRIMARY KEY,
                 doe TEXT,
                 link_id INT,
                 status_code INT,
                 FOREIGN KEY (link_id) REFERENCES wiki_link(link_id))''')
    
    # create table wiki_link_info
    c.execute('''CREATE TABLE IF NOT EXISTS wiki_link_info 
                (link_info_id INTEGER PRIMARY KEY,
                 doe TEXT,
                 link_id INT, 
                 link_size INT,
                 incoming_links INT,
                 coordinates TEXT,
                 page_score INT,
                 first_paragraph TEXT,
                 short_description TEXT,
                 wiki_item TEXT,
                 description TEXT,
                 FOREIGN KEY (link_id) REFERENCES wiki_link(link_id))''')
    
    # create table wiki_link_page_views
    c.execute('''CREATE TABLE IF NOT EXISTS wiki_link_page_views 
                (link_page_views_id INTEGER PRIMARY KEY,
                 doe TEXT,
                 link_id INT,
                 page_views_date TEXT,
                 page_views INT,
                 FOREIGN KEY (link_id) REFERENCES wiki_link(link_id))''')
    
    # create table wiki_image
    c.execute('''CREATE TABLE IF NOT EXISTS wiki_image 
                (image_id INTEGER PRIMARY KEY,
                 doe TEXT,
                 image_url TEXT UNIQUE,
                 image_file TEXT UNIQUE)''')
    
    # create table wiki_image_usage
    c.execute('''CREATE TABLE IF NOT EXISTS wiki_image_usage
                (image_usage_id INTEGER PRIMARY KEY,
                 doe TEXT,
                 image_id INT,
                 link_id INT,
                 FOREIGN KEY (image_id) REFERENCES wiki_image(image_id))''')
   
    # create table wiki license
    c.execute('''CREATE TABLE IF NOT EXISTS wiki_copyright_license
                (license_id INTEGER PRIMARY KEY,
                 doe TEXT,
                 license_name TEXT,
                 license_description TEXT,
                 attrib_required TEXT,
                 copyright TEXT)''')
    
    # create table wiki_user
    c.execute('''CREATE TABLE IF NOT EXISTS wiki_user 
                (user_id INTEGER PRIMARY KEY,
                 doe TEXT,
                 user_name TEXT UNIQUE)''')
 
    
    # create table wiki_image_info
    c.execute('''CREATE TABLE IF NOT EXISTS wiki_image_info 
                (image_info_id INTEGER PRIMARY KEY,
                 doe TEXT,
                 image_id INT,
                 license_id INT,
                 user_id INT,
                 image_repository TEXT,
                 image_date TEXT,
                 image_credit_description TEXT,
                 image_description TEXT,
                 FOREIGN KEY (image_id) REFERENCES wiki_image(image_id),
                 FOREIGN KEY (license_id) REFERENCES wiki_license(license_id),
                 FOREIGN KEY (user_id) REFERENCES wiki_user(user_id))''')

## Database CRUD functions

The process for getting the Wikipedia data for a certain day and saving it into the wikipedia_tdih database consists of three main steps:
- get day data
- get link data
- get image data
    
The CRUD functions are grouped based on which step of the process they occur in.

### 1. Get_day_data related functions
Tables affected:
- wiki_day_data_log
- wiki_link
- wiki_event
- wiki_link_usage

In [ ]:
def add_day_data_to_db(day_data_dict, doe = DOE):
    """
    Main function for adding a day's data to wikipedia_tdih.db.
    It takes the day data retrieved and cleaned up with day_data_main and uses it to update
    the following tables:
        - wiki_day_data_log
        - wiki_link (if link not already in wiki_link)
        - wiki_event        
        - wiki_link_usage
    
    Params:
        day_data_dict: dictionary with day data (created with day_data_main)
        doe: date of entry (defaults to current day)
        
    Returns:
        update_results: a list of dictionaries with the status of each table update
                        e.g. {'doe': doe,
                              'wiki_table_name': 'wiki_event',
                              'update_status': 'update_complete',
                              'update_note': 'events'}
    """
    update_results = []
    
    # make smaller dictionaries based on which table data belongs in
    # data for the wiki_day_data_log
    day_data = {key: day_data_dict[key] for key in ['status_code', 'day_soup', 'no_event_for']}

    # data for wiki_event (and, if needed, wiki_link)
    event_data = {key: day_data_dict.get(key, []) for key in ['events', 'births', 'deaths',
                                                            'holidays_and_observances']}

    
    # get day and month for the data being looged
    day = day_data_dict['day']
    month = day_data_dict['month']
    
    with sqlite3.connect(DATABASE_FILE) as conn:
        c = conn.cursor()
        
        # log data into wiki_day_data_log
        update_wiki_day_data_log = update_table_wiki_day_data_log(day_data, day, month, c, doe)
        update_results.append(update_wiki_day_data_log)
        
        # log data into wiki_event (and, if needed, in wiki_link)
        for event_type in event_data:
            update_wiki_event = update_table_wiki_event(event_data[event_type], event_type, day, month, c, doe)
            update_results.append(update_wiki_event)
            
    return update_results

In [ ]:
def update_table_wiki_day_data_log(day_data, day, month, database_cursor, doe = DOE):
    """
    Add data received from Wikipedia API into the wiki_day_data_log table. 
    This data is minimally processed and is used as restore point in case subsequent CRUD operations
    involving cleaned-up and processed data fail.
    
    Params:
        day_data: dictionary with minimally processed data retrived from Wikipedia for a specific day of the year
                  e.g. {'status_code': status_code_of_the_API_request,
                        'day_soup': : BeautifulSoup_object_representing_day_data,
                        'no_event_for': list_of_event_types_without_events}
        day: day of the events retrieved from Wikipedia
        month: month of the events retrieved from Wikipedia
    
    Returns:
        update_status_dict: dictionary with status of wiki_day_data_log update
                            e.g. {'doe': doe,
                                  'wiki_table_name': 'wiki_day_data_log',
                                  'update_status': update_status
                                  'update_note': np.nan}  # this is relevant to other tables (e.g. wiki_event)
    """
    c = database_cursor
    
    # data to be logged into wiki_day_data_log
    status_code = day_data['status_code']
    day_soup = str(day_data['day_soup']).encode('utf-8') # change BeautifulSoup object to Python blob
    no_event_for = str(day_data['no_event_for'])
    
    # organize the data into a tuple
    data = (doe, day, month, status_code, day_soup, no_event_for)
   
    
    try:
        c.execute('INSERT INTO wiki_day_data_log VALUES (null, ?, ?, ?, ?, ?, ?)', data)
        update_status = 'update_complete'
    except Exception as e:
        update_status = repr(e)
        
    update_status_dict = {'doe': doe,
                          'wiki_table_name': 'wiki_day_data_log',
                          'update_status': update_status,
                          'update_note': np.nan}
        
    return update_status_dict

In [ ]:
def update_table_wiki_event(event_data, event_type, day, month, database_cursor, doe = DOE):
    """
    Add data received from Wikipedia API into the wiki_event table.
    (This data has been cleaned-up and formatted to match the specifications of wikipedia_tdih.db database.)
    
    Params:
        event_data: list of dictionaries with data for the event
                    (eg. {'year': year,
                          'bc_ad': bc_ad,
                          'bc_ad_note': bc_ad_note,
                          'event_description': event_description`,
                          'event_category': event_category, # e.g. 'Christian feast day'
                          'event_first_link': event_first_link,
                          'event_links_list': list_of_links} 
        event_type: event type being processed
                    (i.e. - events
                          - births
                          - deaths
                          - holidays_and_observances)
        day: day of the event
        month: month of the event
        doe: date of entry
        database_cursor: cursor object for wikipedia_tdih.db
       
    Returns:
        update_table_status: a dictionary with the update status for each event type
                             e.g {'event_type': event_type,
                                  'doe': date_of_entry,
                                  'update_status': update_status}
    """
    update_table_status = {}
    status_message = 'no data' # no data was retrieved for event_type 
    
    c = database_cursor
    
    for event in event_data:
        # get the data the needs to be logged
        year = event['year']
        bc_ad = event.get('bc_ad', np.nan) # event_type 'holidays_and_observances' has no bc_ad data
        bc_ad_note = event.get('bc_ad_note', np.nan)
        event_description = event['event_description']
        event_category = event['event_category']
        event_first_link_id = get_wiki_link_id(event['event_first_link'], c, doe)
        event_links_list = event['event_links_list']
        
        # organize the data into a tuple
        data = (doe, day, month, year, bc_ad, bc_ad_note, event_type, event_description,
                event_category, event_first_link_id, str(event_links_list))
        
        # insert data into wiki_event
        c.execute('INSERT INTO wiki_event VALUES (null, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', data)
        
        # update table wiki_link_usage
        event_id = c.lastrowid
        update_table_wiki_link_usage(event_links_list, event['event_first_link'], event_id, c, doe)
        
    status_message = 'update_completed' if event_data else f"no event_type '{event_type}'"

In [ ]:
def update_table_wiki_event(event_data, event_type, day, month, database_cursor, doe = DOE):
    """
    Add data received from Wikipedia API into the wiki_event table.
    (This data has been cleaned-up and formatted to match the specifications of wikipedia_tdih.db database.)
    
    Params:
        event_data: list of dictionaries with data for the event
                    (eg. {'year': year,
                          'bc_ad': bc_ad,
                          'bc_ad_note': bc_ad_note,
                          'event_description': event_description`,
                          'event_category': event_category, # e.g. 'Christian feast day'
                          'event_first_link': event_first_link,
                          'event_links_list': list_of_links} 
        event_type: event type being processed
                    (i.e. - events
                          - births
                          - deaths
                          - holidays_and_observances)
        day: day of the event
        month: month of the event
        database_cursor: cursor object for wikipedia_tdih.db
        doe: date of entry
       
    Returns:
        update_table_status: a dictionary with the update status for each event type
                             e.g {'doe': date_of_entry,
                                  'wiki_table_name': 'wiki_event',
                                  'update_status': update_status,
                                  'event_type': event_type,              
                                  'update_note': f'{event_type}: update_complete'}
    """
    c = database_cursor
    
    for event in event_data:
        # get the data the needs to be logged
        year = event['year']
        bc_ad = event.get('bc_ad', np.nan) # event_type 'holidays_and_observances' has no bc_ad data
        bc_ad_note = event.get('bc_ad_note', np.nan)
        event_description = event['event_description']
        event_category = event['event_category']
        event_first_link_id = get_wiki_link_id(event['event_first_link'], c, doe)
        event_links_list = event['event_links_list']
        
        # organize the data into a tuple
        data = (doe, day, month, year, bc_ad, bc_ad_note, event_type, event_description,
                event_category, event_first_link_id, str(event_links_list))
        
        # insert data into wiki_event
        c.execute('INSERT INTO wiki_event VALUES (null, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', data)
        
        # update table wiki_link_usage
        event_id = c.lastrowid
        update_table_wiki_link_usage(event_links_list, event['event_first_link'], event_id, c, doe)
        
    update_note = event_type if event_data else f'{event_type}: no_data'
    
    update_table_status = {'doe': doe,
                           'wiki_table_name': 'wiki_event',
                           'update_status': 'update_complete',
                           'update_note': update_note}
    
    return update_table_status

In [ ]:
def update_table_wiki_link_usage(event_links_list, event_first_link, event_id, database_cursor, doe = DOE):
    """
    Add link usage info to wiki_link_usage.
    
    Params:
        event_links_list: list of wiki links found in the description of an event
        event_first_link: the first link in the event description (that is not a year_link)
        database_cursor: cursor object for wikipedia_tdih.db
        doe: date of entry (defaults to current day)
        
    Returns:
        None
        
    """
    c = database_cursor
    
    for link in event_links_list:
        # initialize is_first_link
        is_first_link = 0
        # check that link is not the year link (found at the beginning of most events)
        if not is_year_link(link):
            if link == event_first_link:
                is_first_link = 1
            link_id = get_wiki_link_id(link, c, doe)
            
            # data for the wiki_link_usage table
            data = (doe, link_id, event_id, is_first_link)
            
            # update wiki_link_usage table
            c.execute('INSERT INTO wiki_link_usage VALUES (null, ?, ?, ?, ?)', data)

In [ ]:
def is_year_link(wiki_link):
    """
    Check if wiki_link is a year link (in most cases the year links are not relevant to the description 
    of an event. They just link to the wikipedia page for a specific year.)
    
    Params:
        wiki_link: link found in the description of an event
        
    Returns:
        True or False
    """
    # match patterns like: wiki/AD_100, wiki/AD100, wiki/100, wiki/100_AD, wiki/100AD (for AD or BC)
    event_year = re.compile(r'(wiki/ad_*\d{1,4})|(wiki/bc_*\d{1,4})|(wiki/\d{1,4})|(wiki/\d{1,4}_*ad)|(wiki/\d{1,4}_*bc)', re.IGNORECASE)
    
    return event_year.search(wiki_link)

In [ ]:
def get_wiki_link_id(event_link, database_cursor, doe = DOE):
    """
    Get the id of event_link from wiki_link table.
    (If event_link is not in wiki_link, add event_link to wiki_link and return the id of the 
    newly added link)
    
    Params:
        event_link: string representing a wiki_link (e.g. 'wiki/Albert_Einstein')
        database_cursor: cursor object for the wikipedia_tdih.db
        doe: date of entry
          
    Returns:
        link_id    
    """
    c = database_cursor
    link_id = ''
    
    c.execute('SELECT link_id FROM wiki_link WHERE link_url = ?', (event_first_link, ))
    
    try:
        link_id = c.fetchone()[0]
    except:
        pass
    
    if not link_id:
        c.execute('INSERT INTO wiki_link (doe, link_url) VALUES (?, ?)', (doe, event_first_link))
        link_id = c.lastrowid
        
    return link_id

### 2. Get_link_data related functions
Tables affected:
- wiki_link_date_log
- wiki_link_size
- wiki_link_page_views

Flow:
---  
  
1. run day_data_main
    - returns nested dictionary day_data_dict
        - status_code
        - day_text_soup
        - events
        - births
        - deaths
        - holidays_and_observances
        - no_event_for  


2. log data into wiki_get_day_data_log
    - id
    - day
    - month
    - status_code
    - day_text_soup
    - no_event_for